In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GroupKFold, KFold
from sklearn.metrics import log_loss
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OneHotEncoder
import lightgbm as lgb
print(lgb.__version__)
import  time
import os
import tensorflow as tf
from tqdm import tqdm
train_path1 = r"./data/train_fea/20200801.csv"
train_path2 = r"./data/train_fea/20200802.csv"
test_path = r"./data/test_fea/20200901.csv"
submission_path = r"./sample_submission.csv"

submission = pd.read_csv(submission_path)
train_data1 = pd.read_csv(train_path1)
train_data1['date'] = 1

train_data2 = pd.read_csv(train_path2)
train_data2['date'] = 2
train_data = pd.concat([train_data1,train_data2])
test_data =  pd.read_csv(test_path)
test_data['date'] = 3

3.2.1


d:\softinstall\anaconda3\envs\tensorflow13\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
d:\softinstall\anaconda3\envs\tensorflow13\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
d:\softinstall\anaconda3\envs\tensorflow13\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
d:\softinstall\anaconda3\envs\tensorflow13\lib\site-

In [2]:
train_data.head()

,order_id,ata,distance,simple_eta,driver_id,slice_id,weather,hightemp,lowtemp,true_t,wight_t,cross_t,status_t,link_num,cross_num,date
0,5966280,531,2923.0000,495,75457,153,5,28,26,368.0639,441.57001,127.0,129,65,5,1
1,6099942,590,3212.2780,559,19210,157,5,28,26,475.7070,580.64778,83.0,135,63,4,1
2,2792816,559,7868.4474,626,38822,36,5,28,26,596.1600,709.76583,30.0,115,62,1,1
3,5780075,677,4976.3791,446,51627,76,5,28,26,414.2489,497.09868,32.0,140,70,1,1
4,7808558,968,4985.9186,872,39965,52,5,28,26,754.9907,878.61882,117.0,132,75,5,1


+ 长距离的订单只占少数，75%的订单都在4713以下
+ 路长、link_num、和预测指标ata存在着强相关，适合做组合特征

In [3]:
train_data['distance'].describe()

count    148457.000000
mean       3856.073093
std        2850.957727
min          39.693600
25%        2089.206200
50%        3148.491700
75%        4713.768300
max       81475.691700
Name: distance, dtype: float64

In [4]:
train_data['ata'].describe()

count    148457.000000
mean        614.533212
std         357.997692
min          26.000000
25%         368.000000
50%         536.000000
75%         768.000000
max        5456.000000
Name: ata, dtype: float64

In [5]:
train_data['link_num'].describe()


count    148457.000000
mean         63.072870
std          34.758621
min           3.000000
25%          39.000000
50%          56.000000
75%          79.000000
max         501.000000
Name: link_num, dtype: float64

+ 时间特征
+ 将出发时间转化为小时，并进行统计，可以发现大部分人还是喜欢提前9个小时左右提交订单

In [19]:
train_data['slice_hour'] = train_data['slice_id'].apply(lambda x: x*5/60)
#train_data['week_day'] = train_data['date'].apply(lambda x: x%7 + 1)
train_data['slice_hour'].describe()

count    148457.000000
mean          9.584698
std           5.732210
min           0.000000
25%           5.083333
50%           9.416667
75%          13.416667
max          23.916667
Name: slice_hour, dtype: float64

In [21]:
train_data['ata_hour'] = train_data['ata'].apply(lambda x: x/60)
train_data['ata_hour'].describe()

count    148457.000000
mean         10.242220
std           5.966628
min           0.433333
25%           6.133333
50%           8.933333
75%          12.800000
max          90.933333
Name: ata_hour, dtype: float64

+ 前两天有41584个司机

In [ ]:
len(train_data['driver_id'].unique())

